In [ ]:
CSI_Path = ""
Pos_Path = ""
#Path to dataset files

In [ ]:
import h5py
import numpy as np
import os
from sklearn.model_selection import train_test_split

Loading the dataset

In [ ]:
f = h5py.File(CSI_Path , "r")
H_Re, H_Im = f['h_Estimated'][:]
f.close()
f = h5py.File(Pos_Path , "r")
pos = f["r_Position"][:]
f.close()

In [ ]:
H_Mag = H_Re**2 + H_Im**2
del H_Re , H_Im
H_Mag = np.sqrt(H_Mag)
H_Mag = H_Mag.T
pos = pos.T

Creating custom test set

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(H_Mag, pos, test_size=0.1, random_state=42)

In [ ]:
X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],X_train.shape[2],1)

In [ ]:
import tensorflow.keras
from tensorflow.keras.models import Sequential, Model , load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
def create_model():
    '''AutoEncoder Model'''
    model = Sequential()
    model.add(Conv2D(4,(3, 3),activation='tanh',padding='same', input_shape=(16,924,1)))
    model.add(MaxPooling2D((2, 2), padding='same'))
    model.add(Flatten())
    model.add(Dense(3000, activation = 'tanh'))
    model.add(Dense(2000, activation = 'softplus'))
    model.add(Dense(1000, activation = 'tanh'))
    '''Encoded Output'''
    model.add(Dense(500, activation = 'linear'))
    model.add(Dense(1000, activation = 'relu'))
    model.add(Dense(2000, activation = 'softplus'))
    model.add(Dense(3000, activation='tanh'))
    model.add(Dense(14784 , activation='tanh'))
    model.add(Reshape((8,462,4)))
    model.add(UpSampling2D((2, 2)))
    '''Decoded Output'''
    model.add(Conv2D(1,(3, 3),padding='same'))
    model.compile(loss= 'mse', optimizer = Adam(1e-4))
    return model

Training Autoencoder (Channel Charting)

In [ ]:
autoencoder = create_model()
autoencoder.summary()

In [ ]:
earlystopper = EarlyStopping(patience = 25, verbose=1)
checkpointer = ModelCheckpoint('AE_Model_Conv2.h5', verbose=1, save_best_only=True)
autoencoder.fit(X_train,X_train , batch_size=16,epochs=50, validation_split=0.2  ,callbacks=[earlystopper, checkpointer])
autoencoder.fit(X_train,X_train , batch_size=8,epochs=50, validation_split=0.2  ,callbacks=[earlystopper, checkpointer])

Obtaining Channel charts via trained encoder

In [ ]:
autoencoder = load_model("AE_Model_Conv2.h5")
encoder = Model(autoencoder.input,autoencoder.layers[-8].output)
X_tr = encoder.predict(X_train)
X_tr = X_tr.reshape(X_tr.shape[0],X_tr.shape[1],1)
X_tr.shape

In [ ]:
from tensorflow.keras.models import Model,Sequential , load_model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense , Dropout, Conv1D ,Flatten
from tensorflow.keras.optimizers import Adam, SGD

In [ ]:
def classifier():
    ''' Deep CNN to estimate postition'''
    model = Sequential()
    model.add(Conv1D(32,5,input_shape=(100,1)))
    model.add(Flatten())
    model.add(Dropout(0.03))
    model.add(Dense(928 ,activation = 'relu' ))
    model.add(Dropout(0.03))
    model.add(Dense(256 ,activation = 'relu' ))
    model.add(Dense(128 ,activation = 'relu' ))
    model.add(Dropout(0.03))
    model.add(Dense(64 ,activation = 'relu' ))
    model.add(Dropout(0.03))
    model.add(Dense(32 ,activation = 'relu' ))
    model.add(Dropout(0.03))
    model.add(Dense(16 ,activation = 'relu' ))
    model.add(Dropout(0.03))
    model.add(Dense(6 ,activation = 'relu' ))
    model.add(Dropout(0.03))
    model.add(Dense(3))
    model.compile(loss='mean_squared_error', optimizer = Adam(7e-4)) 
    return model

Training network to estimate position

In [ ]:
model = classifier()
model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
earlystopper = EarlyStopping(patience = 50, verbose=1)
checkpointer = ModelCheckpoint('best_AE.h5', verbose=1, save_best_only=True)
hist = model.fit(X_tr,y_train , batch_size=8,epochs=1000, validation_split=0.1  ,callbacks=[earlystopper, checkpointer])
val_loss = hist.history['val_loss']

Evaluating performance on test set

In [ ]:
autoencoder = load_model("AE_Model_Conv.h5")
encoder = Model(autoencoder.input,autoencoder.layers[-8].output)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],X_test.shape[2],1)
X_test = encoder.predict(X_test)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)
X_test.shape
del autoencoder, X_train, H_Mag
X_test.shape

In [ ]:
best = load_model("best_AE.h5")
out = best.predict(X_test)
error = np.sum((out-y_test)**2 , axis=1)
np.mean(np.sqrt(error))

In [ ]:
import matplotlib.pyplot as plt
plt.title("Root Mean Squared Error")
plt.hist(np.sqrt(error),50)

In [ ]:
autoencoder = load_model("AE_Model_Conv.h5")
encoder = Model(autoencoder.input,autoencoder.layers[-9].output)
keras.utils.plot_model(
    encoder,
    to_file="model.png",
    show_shapes=True,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=False,
    dpi=96,
)